Домашнее задание
1. взять любой набор данных для бинарной классификации (можно скачать один из модельных с https://archive.ics.uci.edu/ml/datasets.php)
2. сделать feature engineering
3. обучить любой классификатор (какой вам нравится)
4. далее разделить ваш набор данных на два множества: P (positives) и U (unlabeled). Причем брать нужно не все положительные (класс 1) примеры, а только лишь часть
5. применить random negative sampling для построения классификатора в новых условиях
6. сравнить качество с решением из пункта 4 (построить отчет - таблицу метрик)
7. поэкспериментировать с долей P на шаге 5 (как будет меняться качество модели при уменьшении/увеличении размера P)

AI4I 2020 Профилактическое обслуживание

Информация о наборе данных:

Поскольку реальные наборы данных профилактического обслуживания, как правило, трудно получить и, в частности, трудно опубликовать, мы представляем и предоставляем синтетический набор данных, который, насколько нам известно, отражает реальное профилактическое обслуживание, встречающееся в промышленности.


Информация об атрибутах:

Набор данных состоит из 10000 точек данных, хранящихся в виде строк с 14 функциями в столбцах.
- UID: уникальный идентификатор от 1 до 10000
- Идентификатор продукта: состоит из буквы L, M или H для низкого (50% всех продуктов), среднего (30%) и высокого (20%) как вариантов качества продукта и серийного номера для конкретного варианта.
- температура воздуха [K]: генерируется с использованием процесса случайного блуждания, позже нормализованного до стандартного отклонения 2 K около 300 K
- температура процесса [K]: генерируется с использованием процесса случайного блуждания, нормированного на стандартное отклонение 1 K, добавленного к температуре воздуха плюс 10 K.
- частота вращения [об / мин]: рассчитана для мощности 2860 Вт, с учетом нормально распределенного шума
- крутящий момент [Нм]: значения крутящего момента обычно распределяются около 40 Нм с = 10 Нм и без отрицательных значений.
- износ инструмента [мин]: варианты качества H / M / L добавляют 5/3/2 минуты износа инструмента к используемому инструменту в процессе. 
- machine failure метка «сбой машины», которая указывает, верны ли данные машины в этой конкретной точке данных для любого из следующих режимов сбоя.

Отказ машины состоит из пяти независимых режимов отказа.
- отказ из-за износа инструмента (TWF): инструмент будет заменен в случае отказа при случайно выбранном времени износа инструмента от 200 до 240 минут (120 раз в нашем наборе данных). На данный момент инструмент заменяется 69 раз и выходит из строя 51 раз (назначается случайным образом).
- Отказ отвода тепла (HDF): рассеяние тепла вызывает сбой процесса, если разница между температурой воздуха и технологической среды ниже 8,6 K, а скорость вращения инструмента ниже 1380 об / мин. Это случай 115 точек данных.
- сбой питания (PWF): произведение крутящего момента и скорости вращения (в рад / с) равняется мощности, необходимой для процесса. Если эта мощность ниже 3500 Вт или выше 9000 Вт, процесс завершится ошибкой, что в нашем наборе данных 95 раз.
- отказ от перенапряжения (OSF): если произведение износа инструмента и крутящего момента превышает 11000 минНм для варианта изделия L (12000 M, 13000 H), процесс не выполняется из-за перенапряжения. Это верно для 98 точек данных.
- случайные отказы (RNF): вероятность отказа каждого процесса составляет 0,1% независимо от его параметров процесса. Это справедливо только для 5 точек данных, меньше, чем можно было ожидать для 10 000 точек данных в нашем наборе данных.

Если хотя бы один из вышеперечисленных режимов сбоя истинен, процесс завершается ошибкой, а метка «сбой машины» устанавливается на 1. Следовательно, для метода машинного обучения непрозрачно, какой из режимов сбоя привел к сбою процесса.

In [468]:
import pandas as pd
import numpy as np
data = pd.read_csv("ai4i2020.csv")
data.head(3)

,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,0,0,0,0,0
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,0,0,0,0,0
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,0,0,0,0,0


Посмотрим на соотношение классов

In [469]:
data.iloc[:, -6].value_counts()

0    9661
1     339
Name: Machine failure, dtype: int64

Переместим колонку с меткой о поломке в самый конец dataset 

In [470]:
data = data[['UDI', 'Product ID', 'Type', 'Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]',
            'Torque [Nm]','Tool wear [min]', 'TWF', 'HDF', 'PWF', 'OSF', 'RNF', 'Machine failure']]

In [471]:
data.head(3)

,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],TWF,HDF,PWF,OSF,RNF,Machine failure
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,0,0,0,0,0
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,0,0,0,0,0
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,0,0,0,0,0


Удалим колонки тип и ID Product

In [472]:
df1 = data.pop('Product ID') # remove column ID Product and store it in df1
df2 = data.pop('Type') # remove column Type and store it in df2

Скорректируем имена столбцов из-за наличия в них []

In [473]:
import re
regex = re.compile(r"\[|\]|<", re.IGNORECASE)

In [474]:
data.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in data.columns.values]

In [475]:
data.head(3)

,UDI,Air temperature _K_,Process temperature _K_,Rotational speed _rpm_,Torque _Nm_,Tool wear _min_,TWF,HDF,PWF,OSF,RNF,Machine failure
0,1,298.1,308.6,1551,42.8,0,0,0,0,0,0,0
1,2,298.2,308.7,1408,46.3,3,0,0,0,0,0,0
2,3,298.1,308.5,1498,49.4,5,0,0,0,0,0,0


Разбиваем выборку на тренировочную и тестовую части и обучаем модель (в примере - градиентный бустинг)

In [476]:
from sklearn.model_selection import train_test_split

x_data = data.iloc[:,:-1]
y_data = data.iloc[:,-1]

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=7)

In [477]:
import xgboost as xgb

model = xgb.XGBClassifier(use_label_encoder=False)

model.fit(x_train, y_train)
y_predict = model.predict(x_test)

t1 = y_predict

[13:10:24] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [478]:
from sklearn.metrics import recall_score, precision_score, roc_auc_score, accuracy_score, f1_score

def evaluate_results(y_test, y_predict):
    print('Classification results:')
    f1 = f1_score(y_test, y_predict)
    print("f1: %.2f%%" % (f1 * 100.0)) 
    roc = roc_auc_score(y_test, y_predict)
    print("roc: %.2f%%" % (roc * 100.0)) 
    rec = recall_score(y_test, y_predict, average='binary')
    print("recall: %.2f%%" % (rec * 100.0)) 
    prc = precision_score(y_test, y_predict, average='binary')
    print("precision: %.2f%%" % (prc * 100.0)) 

    
evaluate_results(y_test, y_predict)

Classification results:
f1: 98.70%
roc: 98.72%
recall: 97.44%
precision: 100.00%


### Теперь очередь за PU learning
Представим, что нам неизвестны негативы и часть позитивов

In [479]:
mod_data = data.copy()
#get the indices of the positives samples
pos_ind = np.where(mod_data.iloc[:,-1].values == 1)[0]
#shuffle them
np.random.shuffle(pos_ind)
# leave just 25% of the positives marked
pos_sample_len = int(np.ceil(0.25 * len(pos_ind)))
print(f'Using {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
pos_sample = pos_ind[:pos_sample_len]

Using 85/339 as positives and unlabeling the rest


Создаем столбец для новой целевой переменной, где у нас два класса - P (1) и U (-1)

In [480]:
mod_data['class_test'] = -1
mod_data.loc[pos_sample,'class_test'] = 1
print('target variable:\n', mod_data.iloc[:,-1].value_counts())

target variable:
 -1    9915
 1      85
Name: class_test, dtype: int64


* We now have just 85 positive samples labeled as 1 in the 'class_test' col while the rest is unlabeled as -1.

* Recall that col Machine failure still holds the actual label

In [481]:
mod_data.head(10)

,UDI,Air temperature _K_,Process temperature _K_,Rotational speed _rpm_,Torque _Nm_,Tool wear _min_,TWF,HDF,PWF,OSF,RNF,Machine failure,class_test
0,1,298.1,308.6,1551,42.8,0,0,0,0,0,0,0,-1
1,2,298.2,308.7,1408,46.3,3,0,0,0,0,0,0,-1
2,3,298.1,308.5,1498,49.4,5,0,0,0,0,0,0,-1
3,4,298.2,308.6,1433,39.5,7,0,0,0,0,0,0,-1
4,5,298.2,308.7,1408,40.0,9,0,0,0,0,0,0,-1
5,6,298.1,308.6,1425,41.9,11,0,0,0,0,0,0,-1
6,7,298.1,308.6,1558,42.4,14,0,0,0,0,0,0,-1
7,8,298.1,308.6,1527,40.2,16,0,0,0,0,0,0,-1
8,9,298.3,308.7,1667,28.6,18,0,0,0,0,0,0,-1
9,10,298.5,309.0,1741,28.0,21,0,0,0,0,0,0,-1


Помните, что этот фрейм данных (x_data) включает бывшую целевую переменную, которую мы сохраняем здесь только для сравнения результатов.

[: -2] - это исходная метка класса для положительных и отрицательных данных [: -1] - это новый класс для положительных и немаркированных данных.

In [482]:
x_data = mod_data.iloc[:,:-2].values # just the X 
y_labeled = mod_data.iloc[:,-1].values # new class (just the P & U)
y_positive = mod_data.iloc[:,-2].values # original class

### 1. random negative sampling

In [483]:
mod_data = mod_data.sample(frac=1)
neg_sample = mod_data[mod_data['class_test']==-1][:len(mod_data[mod_data['class_test']==1])]
sample_test = mod_data[mod_data['class_test']==-1][len(mod_data[mod_data['class_test']==1]):]
pos_sample = mod_data[mod_data['class_test']==1]
print(neg_sample.shape, pos_sample.shape)
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

(85, 13) (85, 13)


In [484]:
model = xgb.XGBClassifier()

model.fit(sample_train.iloc[:,:-2].values, 
          sample_train.iloc[:,-2].values)
y_predict = model.predict(sample_test.iloc[:,:-2].values)
evaluate_results(sample_test.iloc[:,-2].values, y_predict)
t2 = y_predict

[13:10:26] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Classification results:
f1: 47.89%
roc: 95.50%
recall: 96.44%
precision: 31.85%


C:\Users\mad_i\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


In [485]:
y_predict

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [489]:
results = pd.DataFrame({'y_true': y_test,
               'Standart XGB': t1})
results

,y_true,Standart XGB
1977,0,0
3880,0,0
52,0,0
2551,0,0
2246,0,0
...,...,...
9505,0,0
2836,0,0
1169,0,0
9929,0,0


In [490]:
from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix

In [491]:
def get_metrics(probs):
    precision, recall, thresholds = precision_recall_curve(y_test, probs)

    fscore = (2 * precision * recall) / (precision + recall)
    # locate the index of the largest f score
    ix = np.argmax(fscore)
    print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f, Roc-AUC=%.3f' % (thresholds[ix], 
                                                                            fscore[ix],
                                                                            precision[ix],
                                                                            recall[ix],
                                                                            roc_auc_score(y_test, probs)))
    return thresholds[ix]

In [492]:
gbm_th = get_metrics(results['Standart XGB'])

Best Threshold=1.000000, F-Score=0.987, Precision=1.000, Recall=0.974, Roc-AUC=0.987


Из за большого дисбаланса в классах метод random negative sampling показал хорошие результаты на полноте, но по точностирезультат не высокий.